In [18]:
import numpy as np
import json
import warnings
import operator

import h5py
from keras.models import model_from_json
from keras import backend as K
from keras.utils import get_custom_objects

from matplotlib import pyplot as plt

warnings.filterwarnings("ignore")

size_title = 18
size_label = 14
n_pred = 2

base_path = "data/evaluate_rnn_compatible_loss_current/"

path_data_dict = base_path + "data_dict.txt"
path_usage_wt = base_path + "usage_prediction.txt"
path_class_wt = base_path + "class_weights.txt"
path_test_data = base_path + "test_paths_dict.txt"
model_path = base_path + "trained_model.hdf5"

def read_file(file_path):
    with open(file_path, 'r') as data_file:
        data = json.loads(data_file.read())
    return data

class_weights = read_file(path_class_wt)
usage_weights = read_file(path_usage_wt)
data_dict = read_file(path_data_dict)


def output_act(class_weights):
    weights = list(class_weights.values())
    def weighted_sigmoid_activation(x):
        exp_weights = K.expand_dims(weights, axis=-1)
        return K.sigmoid(x * K.transpose(exp_weights))
    return weighted_sigmoid_activation


get_custom_objects().update({'weighted_sigmoid_activation': output_act(class_weights)})

def create_model(model_path):
    
    trained_model = h5py.File(model_path, 'r')
    model_config = json.loads(trained_model.get('model_config').value)
    loaded_model = model_from_json(model_config, custom_objects={'activation': output_act(class_weights)})
    dictionary = json.loads(trained_model.get('data_dictionary').value)
    compatibile_tools = json.loads(trained_model.get('compatible_tools').value)
    best_params = json.loads(trained_model.get('best_parameters').value)
    reverse_dictionary = dict((str(v), k) for k, v in dictionary.items())
    model_weights = list()
    weight_ctr = 0
    while True:
        try:
            d_key = "weight_" + str(weight_ctr)
            weights = trained_model.get(d_key).value
            model_weights.append(weights)
            weight_ctr += 1
        except Exception as exception:
            break
    # set the model weights
    loaded_model.set_weights(model_weights)
    return loaded_model, dictionary, reverse_dictionary, compatibile_tools

model, dictionary, reverse_dictionary, compatibile_tools = create_model(model_path)

In [19]:
def analyze_output_layer(model, test_sample, dimensions, phase, iter_num=10):
    output_last = K.function([model.layers[0].input, K.learning_phase()], [model.layers[-1].output])
    result = np.zeros((iter_num,) + (1, dimensions))
    for idx in range(iter_num):
        result[idx] = output_last([test_sample, phase])[0]
    prediction = result.mean(axis=0)
    uncertainty = result.var(axis=0)
    return prediction, uncertainty


def plot_error_bar(prediction, variance):
    pred = prediction[0].tolist()
    var = variance[0].tolist()
    x = range(0, len(pred))
    plt.figure(figsize = (12, 12))
    #plt.errorbar(x, pred, var, marker='s', mfc='red', mec='green', ms=20, mew=4)
    plt.errorbar(x, pred, var, fmt='o', marker='s', mfc='red', mec='green')
    plt.grid(True)
    plt.show()


def verify_model(model, tool_sequence, labels, dictionary, reverse_dictionary, compatible_tools, phase, topk=20, max_seq_len=25):
    tl_seq = tool_sequence.split(",")
    last_tool_name = reverse_dictionary[str(tl_seq[-1])]
    last_compatible_tools = compatible_tools[last_tool_name]
    sample = np.zeros(max_seq_len)
    for idx, tool_id in enumerate(tl_seq):
        sample[idx] = int(tool_id)
    sample_reshaped = np.reshape(sample, (1, max_seq_len))

    tool_sequence_names = [reverse_dictionary[str(tool_pos)] for tool_pos in tool_sequence.split(",")]
    print("Tool seq: %s" % ",".join(tool_sequence_names))
    
    # predict next tools for a test path
    prediction = model.predict(sample_reshaped, verbose=0)
    #prediction, uncertainty = analyze_output_layer(model, sample_reshaped, len(reverse_dictionary) + 1, phase)
    #plot_error_bar(prediction, uncertainty)
    
    prediction = np.reshape(prediction, (prediction.shape[1],))
    prediction_pos = np.argsort(prediction, axis=-1)

    # get topk prediction
    topk_prediction_pos = prediction_pos[-topk:]
    topk_prediction_val = [np.round(prediction[pos], 2) for pos in topk_prediction_pos]

    # read tool names using reverse dictionary
    pred_tool_ids = [reverse_dictionary[str(tool_pos)] for tool_pos in topk_prediction_pos]
    actual_next_tool_ids = list(set(pred_tool_ids).intersection(set(last_compatible_tools.split(","))))      

    print()
    pred_tool_ids_sorted = dict()
    for (tool_pos, tool_pred_val) in zip(topk_prediction_pos, topk_prediction_val):
        tool_name = reverse_dictionary[str(tool_pos)]
        if tool_name in actual_next_tool_ids:
            pred_tool_ids_sorted[tool_name] = tool_pred_val
    pred_tool_ids_sorted = dict(sorted(pred_tool_ids_sorted.items(), key=lambda kv: kv[1], reverse=True))
    
    cls_wt = dict()
    usg_wt = dict()
    inv_wt = dict()
    ids_tools = dict()
    keys = list(pred_tool_ids_sorted.keys())
    for k in keys:
        try:
            cls_wt[k] = np.round(class_weights[str(data_dict[k])], 2)
            usg_wt[k] = np.round(usage_weights[k], 2)
            inv_wt[k] = np.round(inverted_weights[str(data_dict[k])], 2)
        except:
            continue
    print("Predicted tools: \n")
    print(pred_tool_ids_sorted)
    print()
    print("Class weights: \n")
    cls_wt = dict(sorted(cls_wt.items(), key=lambda kv: kv[1], reverse=True))
    print(cls_wt)
    print()
    print("Usage weights: \n")
    usg_wt = dict(sorted(usg_wt.items(), key=lambda kv: kv[1], reverse=True))
    print(usg_wt)
    print()
    total_usage_wt = np.mean(list(usg_wt.values()))
    print("Mean usage wt: %0.4f" % (total_usage_wt))
    print()
    print("Inverted weights: \n")
    inv_wt = dict(sorted(inv_wt.items(), key=lambda kv: kv[1], reverse=True))
    print(inv_wt)
    for key in pred_tool_ids_sorted:
        ids_tools[key] = dictionary[key]
    print()
    print("Tool ids")
    print(ids_tools)
    print("======================================")
    return cls_wt, usg_wt, inv_wt, pred_tool_ids_sorted


In [20]:
reverse_dictionary

{'1': 'chipsequtil_maptoknowngenes',
 '2': 'mothur_summary_seqs',
 '3': 'Digestor',
 '4': 'Nucleosome',
 '5': 'gatk2_variant_apply_recalibration',
 '6': 'deeptools_plot_correlation',
 '7': 'snpeff_to_peptides',
 '8': '__ZIP_COLLECTION__',
 '9': 'rbc_mirdeep2_quantifier',
 '10': 'ggplot2_point',
 '11': 'qiime_make_phylogeny',
 '12': 'abims_CAMERA_annotateDiffreport',
 '13': 'metilene',
 '14': 'snippy_core',
 '15': 'mothur_tree_shared',
 '16': 'egsea',
 '17': 'mothur_lefse',
 '18': 'gatk2_base_recalibrator',
 '19': 'picard_MergeSamFiles',
 '20': 'blast2go',
 '21': 'tp_head_tool',
 '22': 'rbc_splitfasta',
 '23': 'bedtools_bamtofastq',
 '24': 'rnaz',
 '25': 'Add_a_column1',
 '26': 'methtools_dmr',
 '27': 'bcftools_view',
 '28': 'cuffnorm',
 '29': 'wig_to_bigWig',
 '30': 'Psortb',
 '31': 'mothur_classify_seqs',
 '32': 'cshl_grep_tool',
 '33': 'NSPDK_candidateClust',
 '34': 'krona-text',
 '35': 'Cut1',
 '36': 'FeatureLinkerUnlabeled',
 '37': 'cshl_fastx_artifacts_filter',
 '38': 'bedtools_sh

In [28]:
topk = 20
tool_seq = "602,212"
class_wt, usage_wt, inverse_wt, pred_tools = verify_model(model, tool_seq, "", dictionary, reverse_dictionary, compatibile_tools, 1, topk)

Tool seq: trimmomatic,bwa_mem

Predicted tools: 

{'samtools_sort': 0.99, 'bam_to_sam': 0.99, 'samtools_flagstat': 0.99, 'hicexplorer_hicbuildmatrix': 0.98, 'samtools_stats': 0.97, 'samtools_idxstats': 0.96, 'samtools_filter': 0.96, 'samtool_filter2': 0.94, 'picard_MarkDuplicates': 0.94, 'samtools_rmdup': 0.92, 'samtools_mpileup': 0.81, 'deeptools_bam_coverage': 0.81, 'ngsutils_bam_filter': 0.71, 'rseqc_bam2wig': 0.64, 'picard_AddOrReplaceReadGroups': 0.64}

Class weights: 

{'deeptools_bam_coverage': 6.88, 'hicexplorer_hicbuildmatrix': 6.33, 'picard_MarkDuplicates': 5.53, 'samtool_filter2': 5.39, 'picard_AddOrReplaceReadGroups': 5.3, 'samtools_idxstats': 5.22, 'samtools_rmdup': 5.08, 'samtools_flagstat': 4.9, 'samtools_sort': 4.44, 'samtools_mpileup': 4.39, 'bam_to_sam': 4.0, 'samtools_filter': 3.8, 'rseqc_bam2wig': 3.74, 'ngsutils_bam_filter': 3.65, 'samtools_stats': 3.54}

Usage weights: 

{'deeptools_bam_coverage': 1243.06, 'picard_MarkDuplicates': 352.98, 'samtools_flagstat': 274.